# Final Project

## Predict whether a mammogram mass is benign or malignant

We'll be using the "mammographic masses" public dataset from the UCI repository (source: https://archive.ics.uci.edu/ml/datasets/Mammographic+Mass)

This data contains 961 instances of masses detected in mammograms, and contains the following attributes:


   1. BI-RADS assessment: 1 to 5 (ordinal)  
   2. Age: patient's age in years (integer)
   3. Shape: mass shape: round=1 oval=2 lobular=3 irregular=4 (nominal)
   4. Margin: mass margin: circumscribed=1 microlobulated=2 obscured=3 ill-defined=4 spiculated=5 (nominal)
   5. Density: mass density high=1 iso=2 low=3 fat-containing=4 (ordinal)
   6. Severity: benign=0 or malignant=1 (binominal)
   
BI-RADS is an assesment of how confident the severity classification is; it is not a "predictive" attribute and so we will discard it. The age, shape, margin, and density attributes are the features that we will build our model with, and "severity" is the classification we will attempt to predict based on those attributes.

Although "shape" and "margin" are nominal data types, which sklearn typically doesn't deal with well, they are close enough to ordinal that we shouldn't just discard them. The "shape" for example is ordered increasingly from round to irregular.

A lot of unnecessary anguish and surgery arises from false positives arising from mammogram results. If we can build a better way to interpret them through supervised machine learning, it could improve a lot of lives.

## Your assignment

Build a Multi-Layer Perceptron and train it to classify masses as benign or malignant based on its features.

The data needs to be cleaned; many rows contain missing data, and there may be erroneous data identifiable as outliers as well.

Remember to normalize your data first! And experiment with different topologies, optimizers, and hyperparameters.

I was able to achieve over 80% accuracy - can you beat that?


## Let's begin: prepare your data

Start by importing the mammographic_masses.data.txt file into a Pandas dataframe (hint: use read_csv) and take a look at it.

In [1]:
# import necessary libs and modules
import pandas as pd
import numpy as np

from sklearn.preprocessing import StandardScaler

from tensorflow import keras
from tensorflow.keras.layers import Dense
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier

from sklearn.model_selection import cross_val_score

Make sure you use the optional parmaters in read_csv to convert missing data (indicated by a ?) into NaN, and to add the appropriate column names (BI_RADS, age, shape, margin, density, and severity):

In [2]:
df = pd.read_csv("mammographic_masses.data.txt", names=["BI_RADS", "age", "shape", "margin", "density", "severity"], na_values="?")
df

,BI_RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
1,4.0,43.0,1.0,1.0,NaN,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
4,5.0,74.0,1.0,5.0,NaN,1
...,...,...,...,...,...,...
956,4.0,47.0,2.0,1.0,3.0,0
957,4.0,56.0,4.0,5.0,3.0,1
958,4.0,64.0,4.0,5.0,3.0,0
959,5.0,66.0,4.0,5.0,3.0,1


Evaluate whether the data needs cleaning; your model is only as good as the data it's given. Hint: use describe() on the dataframe.

In [3]:
df.describe()

,BI_RADS,age,shape,margin,density,severity
count,959.000000,956.000000,930.000000,913.000000,885.000000,961.000000
mean,4.348279,55.487448,2.721505,2.796276,2.910734,0.463059
std,1.783031,14.480131,1.242792,1.566546,0.380444,0.498893
min,0.000000,18.000000,1.000000,1.000000,1.000000,0.000000
25%,4.000000,45.000000,2.000000,1.000000,3.000000,0.000000
50%,4.000000,57.000000,3.000000,3.000000,3.000000,0.000000
75%,5.000000,66.000000,4.000000,4.000000,3.000000,1.000000
max,55.000000,96.000000,4.000000,5.000000,4.000000,1.000000


There are quite a few missing values in the data set. Before we just drop every row that's missing data, let's make sure we don't bias our data in doing so. Does there appear to be any sort of correlation to what sort of data has missing fields? If there were, we'd have to try and go back and fill that data in.

In [4]:
df[ df.isna().any(axis=1) ]

,BI_RADS,age,shape,margin,density,severity
1,4.0,43.0,1.0,1.0,NaN,1
4,5.0,74.0,1.0,5.0,NaN,1
5,4.0,65.0,1.0,NaN,3.0,0
6,4.0,70.0,NaN,NaN,3.0,0
7,5.0,42.0,1.0,NaN,3.0,0
...,...,...,...,...,...,...
778,4.0,60.0,NaN,4.0,3.0,0
819,4.0,35.0,3.0,NaN,2.0,0
824,6.0,40.0,NaN,3.0,4.0,1
884,5.0,NaN,4.0,4.0,3.0,1


**Answer:** It seems like there are only one or two missing values for each row with NaN values, so it can be assumed that the missing data is at random

If the missing data seems randomly distributed, go ahead and drop rows with missing data. Hint: use dropna().

In [5]:
clean_df = df.dropna()
clean_df

,BI_RADS,age,shape,margin,density,severity
0,5.0,67.0,3.0,5.0,3.0,1
2,5.0,58.0,4.0,5.0,3.0,1
3,4.0,28.0,1.0,1.0,3.0,0
8,5.0,57.0,1.0,5.0,3.0,1
10,5.0,76.0,1.0,4.0,3.0,1
...,...,...,...,...,...,...
956,4.0,47.0,2.0,1.0,3.0,0
957,4.0,56.0,4.0,5.0,3.0,1
958,4.0,64.0,4.0,5.0,3.0,0
959,5.0,66.0,4.0,5.0,3.0,1


Next you'll need to convert the Pandas dataframes into numpy arrays that can be used by scikit_learn. Create an array that extracts only the feature data we want to work with (age, shape, margin, and density) and another array that contains the classes (severity). You'll also need an array of the feature name labels.

In [6]:
X = clean_df[['BI_RADS', 'age', 'shape', 'margin', 'density']].to_numpy()
y = clean_df.severity.values
np.shape(X), len(y)

((830, 5), 830)

Some of our models require the input data to be normalized, so go ahead and normalize the attribute data. Hint: use preprocessing.StandardScaler().

In [7]:
scaler = StandardScaler()
X_transformed = scaler.fit_transform(X)
X_transformed

array([[ 0.3211177 ,  0.7650629 ,  0.17563638,  1.39618483,  0.24046607],
       [ 0.3211177 ,  0.15127063,  0.98104077,  1.39618483,  0.24046607],
       [-0.20875843, -1.89470363, -1.43517241, -1.157718  ,  0.24046607],
       ...,
       [-0.20875843,  0.56046548,  0.98104077,  1.39618483,  0.24046607],
       [ 0.3211177 ,  0.69686376,  0.98104077,  1.39618483,  0.24046607],
       [-0.20875843,  0.42406719,  0.17563638,  0.11923341,  0.24046607]])

## Now build your neural network.

Now set up an actual MLP model using Keras:

**Answer:** Since there are 4 features of each instance, I will have the first layter having 4 neurons. First, let's explore with the good old sigmoid function:

In [8]:
def model1():
    model = keras.Sequential([
        Dense(4, activation="sigmoid"),
        Dense(1, activation="sigmoid"),
    ])
    model.compile(loss='binary_crossentropy', metrics=["accuracy"])
    return model

In [9]:
def model2():
    model = keras.Sequential([
        Dense(4, activation="sigmoid"),
        Dense(4, activation="sigmoid"),
        Dense(1, activation="sigmoid"),
    ])
    model.compile(loss='binary_crossentropy',metrics=["accuracy"])
    return model

In [10]:
def evaluate(model):
    wrapper = KerasClassifier(model, epochs=100, verbose=0)
    scores = cross_val_score(wrapper, X_transformed, y, scoring="accuracy")
    return scores.mean()

In [11]:
evaluate(model1)

0.8192771084337348

In [12]:
evaluate(model2)

0.8192771084337348

Adding another layer of 4 neurons does not seem to have a major impact on the performance of the models. We can experiment with different activation functions to see what works best:

In [13]:
def model3():
    model = keras.Sequential([
        Dense(4, activation="relu"),
        Dense(4, activation="sigmoid"),
        Dense(1, activation="sigmoid"),
    ])
    model.compile(loss='binary_crossentropy',metrics=["accuracy"])
    return model

In [14]:
evaluate(model3)

0.8120481927710843

The model seems to perform a bit better compared to the first one, but the difference is not that significant. I will add another layer with 8 neurons in the hidden layer to see if it has any considerable impact:

In [15]:
def model4():
    model = keras.Sequential([
        Dense(4, activation="relu"),
        Dense(8, activation="relu"),
        Dense(4, activation="sigmoid"),
        Dense(1, activation="sigmoid"),
    ])
    model.compile(loss='binary_crossentropy',metrics=["accuracy"])
    return model

In [16]:
evaluate(model4)

0.8096385542168674

**Observations:** Apparently, this little experiment is not wide enough to check all the combinations of hyperparameters available, so it is too soon to conclude anything. Overall, even with a network of very simple structure, the model managed to achieve over 80% accuracy with the training set. In some runtime cases, this was slightly increased when adding another layer in the middle and also change the activation function of the first layter from Sigmoid to ReLU. However, the this small improvement is insignificant and inconsistent. 

## How did you do?

Which topology, and which choice of hyperparameters, performed the best? Feel free to share your results!

**Answer:** Overall, the it seems like the best hyperparameters combination is the first and simplest one, which has only 1 hidden layer consisting of 4 neurons and the output layer has 1 neuron since this is a binary classification problem.